In [1]:
import tensorflow

In [2]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, GlobalAveragePooling1D, Dense
from keras.datasets import imdb
import numpy as np

In [3]:
max_length=256
padding_type='post'
vocab_size=10000 

In [4]:
(train_data,train_labels),(test_data,test_labels)=imdb.load_data(num_words=vocab_size)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 9s 1us/step


In [5]:
def preprocess_data(data):
    return pad_sequences(data,maxlen=max_length,padding=padding_type)

In [6]:
train_data=preprocess_data(train_data)
test_data=preprocess_data(test_data)

In [7]:
def build_model(vocab_size,embedding_dim=16,hidden_units=16):
    model = Sequential([
        Embedding(vocab_size,embedding_dim,input_length=max_length),
        GlobalAveragePooling1D(),
        Dense(hidden_units,activation='relu'),
        Dense(1,activation='sigmoid')
    ])
    return model

In [8]:
model=build_model(vocab_size)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

C:\Users\gokul\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
history=model.fit(train_data,train_labels,epochs=10,batch_size=32,validation_data=(test_data,test_labels),verbose=2)
test_loss, test_acc=model.evaluate(test_data,test_labels,verbose=2)
print(f"Test Accuracy:{test_acc},Test Loss:{test_loss}")

Epoch 1/10
782/782 - 3s - 4ms/step - accuracy: 0.7332 - loss: 0.5443 - val_accuracy: 0.8318 - val_loss: 0.3851
Epoch 2/10
782/782 - 2s - 3ms/step - accuracy: 0.8756 - loss: 0.3064 - val_accuracy: 0.8700 - val_loss: 0.3112
Epoch 3/10
782/782 - 2s - 3ms/step - accuracy: 0.9014 - loss: 0.2475 - val_accuracy: 0.8818 - val_loss: 0.2877
Epoch 4/10
782/782 - 2s - 3ms/step - accuracy: 0.9144 - loss: 0.2189 - val_accuracy: 0.8856 - val_loss: 0.2799
Epoch 5/10
782/782 - 2s - 3ms/step - accuracy: 0.9261 - loss: 0.1930 - val_accuracy: 0.8789 - val_loss: 0.2924
Epoch 6/10
782/782 - 2s - 3ms/step - accuracy: 0.9368 - loss: 0.1717 - val_accuracy: 0.8634 - val_loss: 0.3339
Epoch 7/10
782/782 - 2s - 3ms/step - accuracy: 0.9388 - loss: 0.1613 - val_accuracy: 0.8692 - val_loss: 0.3257
Epoch 8/10
782/782 - 2s - 3ms/step - accuracy: 0.9434 - loss: 0.1520 - val_accuracy: 0.8663 - val_loss: 0.3392
Epoch 9/10
782/782 - 2s - 3ms/step - accuracy: 0.9489 - loss: 0.1405 - val_accuracy: 0.8715 - val_loss: 0.3357
E

In [10]:
word_index=imdb.get_word_index()
def decode_review(encoded_review):
    reverse_word_index=dict([(value,key) for (key,value) in word_index.items()])
    return ' '.join([reverse_word_index.get(i-3,'?') for i in encoded_review if i>=3])

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 5s 3us/step


In [11]:
def display_incorrect_predictions(test_data,test_labels,predictions,num_examples=3):
    predicted_classes=(predictions>0.5).astype(int)
    incorrect_indices=np.where(predicted_classes.flatten()!=test_labels)[0]
    for i,idx in enumerate(incorrect_indices[:num_examples]):
        print(f"Incorrect Prediction{i+1}:")
        print(f"Review: {decode_review(test_data[idx])}")
        print(f"Actual Sentiment:{'Positive' if test_labels[idx]==1 else 'Negative'}")
        print(f"Predicted Sentiment:{'Positive' if predicted_classes[idx][0]==1 else 'Negative'}")

In [12]:
predictions=model.predict(test_data)
display_incorrect_predictions(test_data,test_labels,predictions)

782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 970us/step
Incorrect Prediction1:
Review: i generally love this type of movie however this time i found myself wanting to kick the screen since i can't do that i will just complain about it this was absolutely idiotic the things that happen with the dead kids are very cool but the alive people are absolute idiots i am a grown man pretty big and i can defend myself well however i would not do half the stuff the little girl does in this movie also the mother in this movie is reckless with her children to the point of neglect i wish i wasn't so angry about her and her actions because i would have otherwise enjoyed the flick what a number she was take my advise and fast forward through everything you see her do until the end also is anyone else getting sick of watching movies that are filmed so dark anymore one can hardly see what is being filmed as an audience we are involved with the actions on the screen so then why the hell can't we have night vision
Act

In [13]:
def predict_and_display_reviews(reviews):
    sequences=[[word_index.get(word,2) for word in review.lower().split()] for review in reviews]
    padded_sequences=preprocess_data(sequences)
    sample_predictions=model.predict(padded_sequences)
    sample_predicted_classes=(sample_predictions>0.5).astype(int)
    for i, review in enumerate(reviews):
        print(f"Review{i+1}:{review}")
        print(f'Predicted Score:{sample_predictions[i]}')
        print(f"Predicted Sentiment:{'Positive' if sample_predicted_classes[i][0]==1 else 'Negative'}")
        print("--------------------------------------------------------------------------------\n")

In [14]:
reviews = [
    "This movie was an excellent portrayal of character development and had stellar acting.",
    "I found the movie to be predictable with a lackluster script.",
    "The cinematography was magnificent, and the pacing was perfect. Highly recommend watching.",
    "It was a terrible movie that wasted two hours of my life. The plot made no sense.",
    "An absolute masterpiece, with a gripping story and profound performances."
]
predict_and_display_reviews(reviews)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Review1:This movie was an excellent portrayal of character development and had stellar acting.
Predicted Score:[0.5903113]
Predicted Sentiment:Positive
--------------------------------------------------------------------------------

Review2:I found the movie to be predictable with a lackluster script.
Predicted Score:[0.4171205]
Predicted Sentiment:Negative
--------------------------------------------------------------------------------

Review3:The cinematography was magnificent, and the pacing was perfect. Highly recommend watching.
Predicted Score:[0.20416282]
Predicted Sentiment:Negative
--------------------------------------------------------------------------------

Review4:It was a terrible movie that wasted two hours of my life. The plot made no sense.
Predicted Score:[0.6483499]
Predicted Sentiment:Positive
--------------------------------------------------------------------------------

Review5:An absolute masterpiece, with a gripping st